In [42]:
import pandas as pd
import numpy as np
#import pickle
from sklearn.model_selection import train_test_split

In [43]:
path_2_train_file = 'C:/Users/chandlerxing/Desktop/Financial Analytics	rain1.csv'
df = pd.read_csv(path_2_train_file)

In [44]:
df.columns

Index(['label', 'id', 'visitTime', 'purchaseTime', 'hour', 'C1', 'C2', 'C3',
       'C4', 'C5', 'C6', 'C7', 'C8', 'N1', 'C9', 'N2', 'N3', 'N4', 'N5', 'N6',
       'N7', 'N8', 'N9', 'N10', 'C10', 'C11', 'C12'],
      dtype='object')

In [45]:
df['C1'].value_counts()

2569487005    8430
2108523568       7
2084406919       6
3438432991       5
256353298        5
              ... 
1427705163       1
4180196685       1
596547023        1
3559230936       1
77004800         1
Name: C1, Length: 22595, dtype: int64

In [46]:
df['C3'].value_counts()

1928325921    1232
3475347077    1212
2780744710    1179
1711803242     936
1822656032     536
              ... 
3540895230       1
2756181513       1
3550862535       1
1199484467       1
1533319149       1
Name: C3, Length: 2863, dtype: int64

In [47]:
df['C10'].value_counts()

3469607743    1425
3458698981      82
3183725626      73
1133005353      71
3662869197      68
              ... 
843115400        1
762569609        1
4142922634       1
830608273        1
487200771        1
Name: C10, Length: 28653, dtype: int64

In [48]:
label = df['label'].values
label

array([-1, -1, -1, ..., -1, -1, -1])

In [49]:
data = df.drop(columns=['label','id','visitTime','purchaseTime','C1','C3','C10']).values
data

array([[         0, 1830830742,  652481754, ...,          0, 3617851689,
        4220791646],
       [         0,  403381353,  652481754, ...,          0, 3831440054,
        1213938795],
       [         0,  403381353,  652481754, ...,          0, 2379633508,
        1284343215],
       ...,
       [        23, 1235983246,  652481754, ...,          0,  621902171,
        3706004275],
       [        23,  403381353,  652481754, ...,          0, 3617851689,
        4220791646],
       [        23,  403381353,  652481754, ...,          0, 3617851689,
        4220791646]])

In [50]:
X = data.copy()
y = np.array(label.copy())
y[:5]

array([-1, -1, -1, -1, -1])

In [51]:
y = (y > 0)*1
nb_class = len(set(y))
print(nb_class) 
id_Train = np.array([None]*nb_class)
id_Test = np.array([None]*nb_class)
for i in range(nb_class):
    id_i = np.where(y==i)[0]
    id_i_train, id_i_test = train_test_split(id_i, test_size=0.3, random_state=123,shuffle=True)
    id_Train[i] = id_i_train
    id_Test[i] = id_i_test

id_Train = np.concatenate(id_Train)
id_Test = np.concatenate(id_Test)

X_train = X[id_Train]
Y_train = y[id_Train]
X_test = X[id_Test]
Y_test = y[id_Test]

2


In [52]:
X_train[0]

array([         9, 1830830742,  306394936, 2967512603, 2712499304,
       2303407174, 1804593680,          0,  385980380,          0,
                0,          0,          0,          0,          0,
                0,          0,          0,  621902171,  830706566])

In [53]:
id_Train

array([10261, 17599, 25347, ..., 18397,  2470, 24118])

In [54]:
id_toTrain = np.array([np.where(Y_train==i)[0] for i in range(nb_class)])

size_max = [len(id_toTrain[i]) for i in range(nb_class)]
print("before resampling")
print(size_max)

blc = 150
for i in range(len(size_max)):
    if size_max[i] > blc:
        size_max[i] = int(blc*(np.log10(size_max[i]/blc)+1))
    else:
        size_max[i] = int(blc/(np.log10(blc/size_max[i])+1))

print("after resampling")
print(size_max)
for i in range(nb_class):
    if len(id_toTrain[i]) > size_max[i]:
        id_toTrain[i], tmp = train_test_split(id_toTrain[i], test_size=1-size_max[i]/len(id_toTrain[i]))
    else:
        id_toTrain[i] = np.concatenate((id_toTrain[i], id_toTrain[i][np.random.randint(len(id_toTrain[i]), size=int(size_max[i]-len(id_toTrain[i])))]))
id_toTrain = np.concatenate(id_toTrain)
X_toTrain = X_train[id_toTrain]
Y_toTrain = Y_train[id_toTrain]

before resampling
[21959, 39]
after resampling
[474, 94]


In [55]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score as acs
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import KNNImputer

print('Training')
# evaluate each model in turn
results = []
names = []
rf = RandomForestClassifier(max_depth = 15, random_state=123,max_leaf_nodes=60)
rf = rf.fit(X_toTrain, Y_toTrain)
train_pred = rf.predict(X_train)
print(metrics.accuracy_score(Y_train,train_pred))
print(metrics.classification_report(Y_train,train_pred))


print('\n')

print('Testing')


y_pred = rf.predict(X_test)
#Evaluate predictions
print(metrics.accuracy_score(Y_test,y_pred))
print(metrics.classification_report(Y_test,y_pred))

# Get numerical feature importances
feature_list = X
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Training
0.988953541231021
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     21959
           1       0.14      1.00      0.24        39

    accuracy                           0.99     21998
   macro avg       0.57      0.99      0.62     21998
weighted avg       1.00      0.99      0.99     21998



Testing
0.988441145281018
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      9412
           1       0.13      0.89      0.23        18

    accuracy                           0.99      9430
   macro avg       0.56      0.94      0.61      9430
weighted avg       1.00      0.99      0.99      9430



TypeError: unsupported format string passed to numpy.ndarray.__format__

In [ ]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.002
selector = SelectFromModel(rf, threshold=0.001)

# Train the selector
selector.fit(X_toTrain, Y_toTrain)

In [56]:
# Print the names of the most important features
my_list = []

for feature_list_index in selector.get_support(indices=True):
    #print(X.columns[feature_list_index])
    my_list.append(X[feature_list_index])

In [57]:
X_important_train = selector.transform(X_toTrain)
X_important_test = selector.transform(X_test)

# New Random Forest model.
clf_important = RandomForestClassifier(max_depth = 12, random_state=123, n_estimators = 500 ,max_leaf_nodes = 15)

# Train the new classifier.
clf_important.fit(X_important_train, Y_toTrain)
y_important_pred = clf_important.predict(X_important_train)
print('train')
print(metrics.accuracy_score(Y_toTrain,y_important_pred))

print(metrics.classification_report(Y_toTrain,y_important_pred))
print('test')
y_important_test_pred = clf_important.predict(X_important_test)
print(metrics.accuracy_score(Y_test,y_important_test_pred))
print(metrics.classification_report(Y_test,y_important_test_pred))

train
0.9859154929577465
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       474
           1       0.99      0.93      0.96        94

    accuracy                           0.99       568
   macro avg       0.99      0.96      0.97       568
weighted avg       0.99      0.99      0.99       568

test
0.9937433722163309
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      9412
           1       0.22      0.89      0.35        18

    accuracy                           0.99      9430
   macro avg       0.61      0.94      0.67      9430
weighted avg       1.00      0.99      1.00      9430



In [58]:
# Save the model.
Rfmodel = 'C:/Users/chandlerxing/Desktop/Financial Analytics/model_trained.sav'
pickle.dump(clf_important, open(Rfmodel, 'wb'))

In [59]:
# load the model.
loaded_model = pickle.load(open(Rfmodel, 'rb'))

In [60]:
dft = pd.read_csv("C:/Users/chandlerxing/Desktop/Financial Analytics/test1.csv")

In [61]:
labelt = dft['label'].values
labelt

array([-1, -1, -1, ..., -1, -1, -1])

In [62]:
datat = dft.drop(columns=['label','id','visitTime','purchaseTime','C1','C3','C10','C11']).values
datat

array([[         0,  403381353,  652481754, ...,          0,          0,
        4220791646],
       [         0, 1235983246,  652481754, ...,          0,          0,
        1284343215],
       [         0, 1235983246,  652481754, ...,          0,          0,
        1213938795],
       ...,
       [        23,  403381353,  652481754, ...,          0,          0,
        4220791646],
       [        23, 2541203883, 2308170622, ...,          0,          0,
        4220791646],
       [        23,  403381353,  652481754, ...,          0,          0,
        4220791646]])

In [64]:
dft['prob0'] = loaded_model.predict_proba(datat)[0:,0]
dft = dft[['id','prob0']]
dft.to_csv('C:/Users/chandlerxing/Desktop/Financial Analytics/Predict.csv')